In [2]:
from sqlalchemy import *
import mediacloud, datetime

### Creating a Table
Read more about [defining and creating tables](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#define-and-create-tables).

In [3]:
# add `echo=True` to see log statements of all the SQL that is generated
echo=True
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
metadata = MetaData()
# define a table to use
queries = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('keywords', String(400), nullable=False),
    Column('timestamp', DateTime, default=datetime.datetime.now),
    Column('sentencecount', Integer),
)
metadata.create_all(engine) # and create the tables in the database

### Insert Data
Read more about generating [SQL insert statements](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#insert-expressions).

In [4]:
insert_stmt = queries.insert()
str(insert_stmt) # see an example of what this will do

'INSERT INTO queries (id, keywords, timestamp, sentencecount) VALUES (:id, :keywords, :timestamp, :sentencecount)'

In [5]:
insert_stmt = queries.insert().values(keywords="puppies")
str(insert_stmt) # see an example with a value in the keywords field

'INSERT INTO queries (keywords, timestamp) VALUES (:keywords, :timestamp)'

In [6]:
query = "black lives matter"
mc = mediacloud.api.MediaCloud('136e9fa634cc38b17a3e51b9095c2b3ee1e5045de77ece72f3de4e58df810543')
res = mc.sentenceCount(query, solr_filter=[mc.publish_date_query( datetime.date( 2015, 1, 1), datetime.date( 2015, 11, 12) ), 'media_sets_id:1' ])
print res['count'] # prints the number of sentences found
count = res['count']
insert_stmt = queries.insert().values(keywords=query, sentencecount=count) # prep the insert statement for the next step

590608


In [7]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt) # execute the insert command for the above stmt
result.inserted_primary_key # print out the primary key it was assigned

[1]

In [8]:
query = "all lives matter"
res = mc.sentenceCount(query, solr_filter=[mc.publish_date_query( datetime.date( 2015, 1, 1), datetime.date( 2015, 11, 12) ), 'media_sets_id:1' ])
print res['count'] # prints the number of sentences found
count = res['count']
insert_stmt = queries.insert().values(keywords=query, sentencecount=count) # prep the second query

1878614


In [9]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt) # execute the insert
result.inserted_primary_key # print out the primary key it was assigned

[2]

### Count the sentences across the 2 queries
Read more about using [SQL select statments](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#selecting).

In [10]:
from sqlalchemy.sql import select
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    print row # just make sure they are there and confirm what they look like

(1, u'black lives matter', datetime.datetime(2015, 12, 2, 21, 54, 27, 143552), 590608)
(2, u'all lives matter', datetime.datetime(2015, 12, 2, 21, 55, 1, 274882), 1878614)


In [11]:
select_stmt = select([queries]).where(queries.c.sentencecount)
sum = 0
for row in db_conn.execute(select_stmt):
    print row.sentencecount
    sum = sum + row.sentencecount
print "total sentences returned by all queries is %i" %sum

590608
1878614
total sentences returned by all queries is 2469222
